In [ ]:
# Create a virtual environment
#!python3 -m venv dc_env

# Activate the virtual environment
#import os
#os.system('source dc_env/bin/activate')
#pip install ipykernel
#python -m ipykernel install --user --name=dc_env --display-name "Python (dc_env)"

# Install required packages
!pip install depthcharge-ms
!pip install pylance==0.15


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
import depthcharge as dc
from depthcharge.encoders import PeakEncoder
from depthcharge.data import SpectrumDataset, StreamingSpectrumDataset
import torch

This does the preprocessing of the data to ensure that the noise isn't lost. It outputs a dataframe that we then need to encode.

In [18]:
mzml = './orbitrap_lumos/08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML'
processing_fn = [
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.set_mz_range
    dc.data.preprocessing.set_mz_range(min_mz=0),
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.filter_intensity
    dc.data.preprocessing.filter_intensity(min_intensity = 0.0),
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.scale_intensity
    dc.data.preprocessing.scale_intensity(scaling=None),  # Might want to change later
    dc.data.preprocessing.scale_to_unit_norm,  # Not sure what this is yet
]
df = dc.data.spectra_to_df(
    mzml,
    progress=True,
    preprocessing_fn=processing_fn
)
#df = dc.data.spectra_to_df(mzml, progress=True)

08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML: 100%|██████████| 54852/54852 [00:22<00:00, 2399.95 spectra/s]


In [4]:
print(df.head())

shape: (5, 7)
┌──────────────┬──────────────┬──────────┬──────────────┬──────────────┬─────────────┬─────────────┐
│ peak_file    ┆ scan_id      ┆ ms_level ┆ precursor_mz ┆ precursor_ch ┆ mz_array    ┆ intensity_a │
│ ---          ┆ ---          ┆ ---      ┆ ---          ┆ arge         ┆ ---         ┆ rray        │
│ str          ┆ str          ┆ u8       ┆ f64          ┆ ---          ┆ list[f64]   ┆ ---         │
│              ┆              ┆          ┆              ┆ i16          ┆             ┆ list[f64]   │
╞══════════════╪══════════════╪══════════╪══════════════╪══════════════╪═════════════╪═════════════╡
│ 08CPTAC_C_GB ┆ controllerTy ┆ 2        ┆ 702.865723   ┆ 2            ┆ [143.360748 ┆ [0.011069,  │
│ M_W_PNNL_202 ┆ pe=0 control ┆          ┆              ┆              ┆ ,           ┆ 0.011332, … │
│ 10830_…      ┆ lerNum…      ┆          ┆              ┆              ┆ 146.805038, ┆ 0.01651…    │
│              ┆              ┆          ┆              ┆              ┆ … 70

In [5]:
encoder = PeakEncoder(100)

In [25]:
dataset = StreamingSpectrumDataset(df, batch_size=20)

In [26]:
count = 0
for spectrum in dataset:
    # embeded = encoder.forward(spectrum["mz_array"])
    a_spec = spectrum
    #if (count == 0):
    #    break  
    count += 1
print(count)

1


In [27]:
mz_values = a_spec["mz_array"][0]
intensities = a_spec["intensity_array"][0]
peaks = list(zip(mz_values, intensities))


In [28]:
mz_values[0]

tensor(120.1863)

In [29]:
test1 = torch.tensor([peaks])

In [30]:
encoded_test1 = encoder.forward(test1)

In [31]:
encoded_test1.shape

torch.Size([1, 2077, 100])

In [ ]:
from torch.utils.data import DataLoader
from depthcharge.transformers import SpectrumTransformerEncoder

d_model = 128
model = SpectrumTransformerEncoder(d_model=d_model, 
                                   nhead=8, 
                                   dim_feedforward=1024, 
                                   n_layers=1, 
                                   dropout=0.0, 
                                   peak_encoder= PeakEncoder(d_model=d_model, 
                                                             min_mz_wavelength=0.001, 
                                                             max_mz_wavelength=10000, 
                                                             min_intensity_wavelength=0, 
                                                             max_intensity_wavelength=1, 
                                                             learnable_wavelengths=False) )
# Grab the first 1000 spectra for testing
#b_size = 1000
# loader = DataLoader(dataset, batch_size=None)
# batch1 = next(dataset)
for batch in dataset:
    out = model(batch["mz_array"], batch["intensity_array"])
    print(out[0].shape)
    #break
#for batch in loader:
#    print(batch["scan_id"], batch["precursor_mz"])

torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201, 128])
torch.Size([1000, 201, 128])
torch.Size([24, 201

In [14]:
from depthcharge.transformers import SpectrumTransformerEncoder

model = SpectrumTransformerEncoder()

for spectrum in batch1:
    out = model(spectrum["mz_array"], spectrum["intensity_array"])
    print(out[0].shape)

NameError: name 'batch1' is not defined

In [15]:
a_spec


{'peak_file': ['08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML',
  '08CP